<a href="https://colab.research.google.com/github/prateekbeh/algo_ds/blob/master/seismic_lwe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset as dt
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.dates as mdt
import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar
from datetime import datetime
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.animation as anim
from shapely.geometry import Point , shape , Polygon
from dataclasses import dataclass, fields
import matplotlib.pyplot as pl
import hvplot.xarray
#import fiona
import shapefile
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
import netCDF4 as nc
import math
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from datetime import date
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, Point

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 70.0 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the catalog data from Google Drive
file_path = '/content/drive/MyDrive/grace_data/catalog.txt'
catalog_data = pd.read_csv(file_path, delim_whitespace=True)

# OLD CODE START

# OLD CODE END

# New Step 1: Convert the date column to datetime format
catalog_data['date'] = pd.to_datetime(catalog_data['date'], format='%Y-%m-%d')

# Filter seismic data starting from 2022-04
catalog_data = catalog_data[catalog_data['date'] >= '2022-04-01']

# OLD CODE START
# New Step 2: Divide the dataset into a grid of 1 x 1 degree
# Round the latitude and longitude to the nearest 1 to create 1 x 1 degree grids
catalog_data['lat_grid'] = (catalog_data['latitude'] // 1 * 1).round(1)
# OLD CODE END

# New Step 2: Divide the dataset into a grid of 0.5 x 0.5 degree
# Round the latitude and longitude to the nearest 0.25 or 0.75
catalog_data['lat_grid'] = (catalog_data['latitude'] * 2).round() / 2
catalog_data['lon_grid'] = (catalog_data['longitude'] * 2).round() / 2

# New Step 3: Group by lat_grid, lon_grid, and month to get the sum of events per month
catalog_data['year_month'] = catalog_data['date'].dt.to_period('M')
events_per_month = catalog_data.groupby(['lat_grid', 'lon_grid', 'year_month']).agg(
    event_count=('year_month', 'size'),
    lat_list=('latitude', lambda x: list(x)),
    lon_list=('longitude', lambda x: list(x)),
    max_mag=('mag', 'max'),
    min_mag=('mag', 'min'),
    avg_mag=('mag', 'mean')
).reset_index()

# New Step 4: Sort the result by year_month
events_per_month = events_per_month.sort_values(by='year_month')

# Step 5: Count the number of unique grid points for each month
unique_grid_points_per_month = events_per_month.groupby('year_month')[['lat_grid', 'lon_grid']].nunique().reset_index()
unique_grid_points_per_month.rename(columns={'lat_grid': 'unique_lat_grid_count', 'lon_grid': 'unique_lon_grid_count'}, inplace=True)

# Display the number of unique grid points for each month
print("Number of unique grid points for each month:")
print(unique_grid_points_per_month)

# Step 6: Get the grid with the maximum number of events for 01-01-2020
max_event_date = '2020-01'
events_2020_01 = events_per_month[events_per_month['year_month'] == max_event_date]
max_event_grid = events_2020_01[events_2020_01['event_count'] == events_2020_01['event_count'].max()]

# Display the grid with the maximum number of events for January 2020
print("Grid with the maximum number of events for January 2020:")
print(max_event_grid[['lat_grid', 'lon_grid', 'event_count', 'lat_list', 'lon_list']])

# Display the result
events_per_month.head()

# New Step 7: Read a NetCDF file (GRACE Mascon data)
from netCDF4 import Dataset

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the GRACE Mascon data from Google Drive
netcdf_file_path = '/content/drive/MyDrive/grace_data/GRACE_Mascon_data.nc'
with Dataset(netcdf_file_path, mode='r') as nc_file:
    # List all variables in the NetCDF file
    print("Variables in GRACE Mascon NetCDF file:")
    print(nc_file.variables.keys())

    # Extract data for specific variables (example: lwe_thickness)
    lwe_thickness = nc_file.variables['lwe_thickness'][:]
    latitudes = nc_file.variables['lat'][:]
    longitudes = nc_file.variables['lon'][:]
    times = nc_file.variables['time'][:]

    # Display basic information about the extracted data
    print("LWE Thickness Data Shape:", lwe_thickness.shape)
    print("Latitude Data Shape:", latitudes.shape)
    print("Longitude Data Shape:", longitudes.shape)
    print("Time Data Shape:", times.shape)

    # Step 8: Filter GRACE data for grid points present in catalog data for that particular month
    grace_filtered_data = []
    for index, row in events_per_month.iterrows():
        lat_grid, lon_grid, year_month = row['lat_grid'], row['lon_grid'], row['year_month']
        # Find matching lat/lon in GRACE data
        lat_match = np.isclose(latitudes, lat_grid, atol=0.5)
        lon_match = np.isclose(longitudes, lon_grid, atol=0.5)
        if np.any(lat_match) and np.any(lon_match):
            lat_idx = np.where(lat_match)[0][0]
            lon_idx = np.where(lon_match)[0][0]
            time_idx = np.where(times == year_month.to_timestamp().toordinal())[0]
            if len(time_idx) > 0:
                grace_filtered_data.append({
                    'lat_grid': lat_grid,
                    'lon_grid': lon_grid,
                    'year_month': year_month,
                    'lwe_thickness': lwe_thickness[time_idx[0], lat_idx, lon_idx]
                })

    # Convert the filtered GRACE data to DataFrame
    grace_filtered_df = pd.DataFrame(grace_filtered_data)
    print("Filtered GRACE Data:")
    print(grace_filtered_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-4-50fae4a7024d>:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  catalog_data = pd.read_csv(file_path, delim_whitespace=True)
<ipython-input-4-50fae4a7024d>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  catalog_data['lat_grid'] = (catalog_data['latitude'] // 1 * 1).round(1)
<ipython-input-4-50fae4a7024d>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  catalog_data['lat_grid'] = (catalog_da

Number of unique grid points for each month:
   year_month  unique_lat_grid_count  unique_lon_grid_count
0     2022-04                     10                      7
1     2022-05                     12                     13
2     2022-06                     10                     13
3     2022-07                     15                     11
4     2022-08                     14                     12
5     2022-09                     15                     14
6     2022-10                     15                     12
7     2022-11                      9                      8
8     2022-12                     15                     13
9     2023-01                     11                     16
10    2023-02                      7                      6
11    2023-03                     11                     10
12    2023-04                     12                     13
13    2023-05                     14                     12
14    2023-06                     11                   

In [ ]:
from netCDF4 import Dataset

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the GRACE Mascon data from Google Drive
netcdf_file_path = '/content/drive/MyDrive/grace_data/GRACE_Mascon_data.nc'
with Dataset(netcdf_file_path, mode='r') as nc_file:
    # List all variables in the NetCDF file
    print("Variables in GRACE Mascon NetCDF file:")
    print(nc_file.variables.keys())

    # Extract data for specific variables (example: lwe_thickness)
    lwe_thickness = nc_file.variables['lwe_thickness'][:]
    latitudes = nc_file.variables['lat'][:]
    longitudes = nc_file.variables['lon'][:]
    times = nc_file.variables['time'][:]

    # Display basic information about the extracted data
    print("LWE Thickness Data Shape:", lwe_thickness.shape)
    print("Latitude Data Shape:", latitudes.shape)
    print("Longitude Data Shape:", longitudes.shape)
    print("Time Data Shape:", times.shape)

    # Step 8: Add year_month to GRACE data
from datetime import datetime, timedelta

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Create a DataFrame for GRACE data
# Loop through each time index to create entries for each grid at each time step
grace_data_list = []
for time_idx, year_month in enumerate(grace_year_month):
    for lat_idx, latitude in enumerate(latitudes):
        for lon_idx, longitude in enumerate(longitudes):
            lwe_value = lwe_thickness[time_idx, lat_idx, lon_idx]
            grace_data_list.append({
                'year_month': year_month,
                'latitude': latitude,
                'longitude': longitude,
                'lwe_thickness': lwe_value
            })

# Convert list of dictionaries to DataFrame
grace_data = pd.DataFrame(grace_data_list)

# Step 9: Filter GRACE data for grid points present in catalog data for the common year_month
grace_filtered_data = grace_data.merge(events_per_month[['lat_grid', 'lon_grid', 'year_month']],
                                       left_on=['latitude', 'longitude', 'year_month'],
                                       right_on=['lat_grid', 'lon_grid', 'year_month'],
                                       how='inner')

# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness']]
print("Filtered GRACE Data:")
print(grace_filtered_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Variables in GRACE Mascon NetCDF file:
dict_keys(['lon', 'lat', 'time', 'land_mask', 'lwe_thickness', 'lon_bounds', 'lat_bounds', 'time_bounds'])
LWE Thickness Data Shape: (231, 360, 720)
Latitude Data Shape: (360,)
Longitude Data Shape: (720,)
Time Data Shape: (231,)


In [ ]:
# prompt: I want to get the same grid points in the GRACE data for a particular year_month as in the events_per_month data. update the code for it

    # Step 8: Filter GRACE data for grid points present in catalog data for that particular month
    grace_filtered_data = []
    for index, row in events_per_month.iterrows():
        lat_grid, lon_grid, year_month = row['lat_grid'], row['lon_grid'], row['year_month']
        # Find matching lat/lon in GRACE data
        lat_match = np.isclose(latitudes, lat_grid, atol=0.5)
        lon_match = np.isclose(longitudes, lon_grid, atol=0.5)

        # Check if any match is found before proceeding
        if np.any(lat_match) and np.any(lon_match):
            lat_idx = np.where(lat_match)[0][0]
            lon_idx = np.where(lon_match)[0][0]

            # Convert year_month to ordinal for comparison with GRACE time
            year_month_ordinal = year_month.to_timestamp().toordinal()

            # Find the closest time index in GRACE data to the current year_month
            time_diffs = np.abs(times - year_month_ordinal)
            time_idx = np.argmin(time_diffs)

            grace_filtered_data.append({
                'lat_grid': lat_grid,
                'lon_grid': lon_grid,
                'year_month': year_month,
                'lwe_thickness': lwe_thickness[time_idx, lat_idx, lon_idx]
            })

    # Convert the filtered GRACE data to DataFrame
    grace_filtered_df = pd.DataFrame(grace_filtered_data)
    print("Filtered GRACE Data:")
grace_filtered_df

IndentationError: unexpected indent (<ipython-input-1-b3e65b1b505b>, line 4)

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the catalog data from Google Drive
file_path = '/content/drive/MyDrive/grace_data/catalog.txt'
catalog_data = pd.read_csv(file_path, delim_whitespace=True)

# OLD CODE START

# OLD CODE END

# New Step 1: Convert the date column to datetime format
catalog_data['date'] = pd.to_datetime(catalog_data['date'], format='%Y-%m-%d')

# Filter seismic data starting from 2022-04
catalog_data = catalog_data[catalog_data['date'] >= '2002-04-01']


# New Step 2: Divide the dataset into a grid of 0.5 x 0.5 degree
# Round the latitude and longitude to the nearest 0.25 or 0.75
catalog_data['lat_grid'] = (catalog_data['latitude'] * 2).round() / 2
catalog_data['lon_grid'] = (catalog_data['longitude'] * 2).round() / 2

# New Step 3: Group by lat_grid, lon_grid, and month to get the sum of events per month
catalog_data['year_month'] = catalog_data['date'].dt.to_period('M')
events_per_month = catalog_data.groupby(['lat_grid', 'lon_grid', 'year_month']).agg(
    event_count=('year_month', 'size'),
    lat_list=('latitude', lambda x: list(x)),
    lon_list=('longitude', lambda x: list(x)),
    max_mag=('mag', 'max'),
    min_mag=('mag', 'min'),
    avg_mag=('mag', 'mean')
).reset_index()

# New Step 4: Sort the result by year_month
events_per_month = events_per_month.sort_values(by='year_month')

# Step 5: Count the number of unique grid points for each month
unique_grid_points_per_month = events_per_month.groupby('year_month')[['lat_grid', 'lon_grid']].nunique().reset_index()
unique_grid_points_per_month.rename(columns={'lat_grid': 'unique_lat_grid_count', 'lon_grid': 'unique_lon_grid_count'}, inplace=True)

# Display the number of unique grid points for each month
print("Number of unique grid points for each month:")
print(unique_grid_points_per_month)

# Step 6: Get the grid with the maximum number of events for 01-01-2020
max_event_date = '2020-01'
events_2020_01 = events_per_month[events_per_month['year_month'] == max_event_date]
max_event_grid = events_2020_01[events_2020_01['event_count'] == events_2020_01['event_count'].max()]

# Display the grid with the maximum number of events for January 2020
print("Grid with the maximum number of events for January 2020:")
print(max_event_grid[['lat_grid', 'lon_grid', 'event_count', 'lat_list', 'lon_list']])

# Display the result
events_per_month.head()

# New Step 7: Read a NetCDF file (GRACE Mascon data)
from netCDF4 import Dataset

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the GRACE Mascon data from Google Drive
netcdf_file_path = '/content/drive/MyDrive/grace_data/GRACE_Mascon_data.nc'
with Dataset(netcdf_file_path, mode='r') as nc_file:
    # List all variables in the NetCDF file
    print("Variables in GRACE Mascon NetCDF file:")
    print(nc_file.variables.keys())

    # Extract data for specific variables (example: lwe_thickness)
    lwe_thickness = nc_file.variables['lwe_thickness'][:]
    latitudes = nc_file.variables['lat'][:]
    longitudes = nc_file.variables['lon'][:]
    times = nc_file.variables['time'][:]

    # Display basic information about the extracted data
    print("LWE Thickness Data Shape:", lwe_thickness.shape)
    print("Latitude Data Shape:", latitudes.shape)
    print("Longitude Data Shape:", longitudes.shape)
    print("Time Data Shape:", times.shape)

    # Step 8: Add year_month to GRACE data
from datetime import datetime, timedelta

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Step 8: Filter GRACE data for California region only
california_lat_min, california_lat_max = 32, 42
california_lon_min, california_lon_max = -125, -114

# Filter GRACE data for California latitudes and longitudes
lat_indices = np.where((latitudes >= california_lat_min) & (latitudes <= california_lat_max))[0]
lon_indices = np.where((longitudes >= california_lon_min) & (longitudes <= california_lon_max))[0]

# Extract the filtered data
filtered_latitudes = latitudes[lat_indices]
filtered_longitudes = longitudes[lon_indices]
filtered_lwe_thickness = lwe_thickness[:, lat_indices, :][:, :, lon_indices]

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Create a DataFrame for filtered GRACE data
filtered_grace_data_list = []
for time_idx, year_month in enumerate(grace_year_month):
    for lat_idx, latitude in enumerate(filtered_latitudes):
        for lon_idx, longitude in enumerate(filtered_longitudes):
            lwe_value = filtered_lwe_thickness[time_idx, lat_idx, lon_idx]
            filtered_grace_data_list.append({
                'year_month': year_month,
                'latitude': latitude,
                'longitude': longitude,
                'lwe_thickness': lwe_value
            })

# Convert list of dictionaries to DataFrame
grace_data = pd.DataFrame(filtered_grace_data_list)
# Correct DataFrame creation for filtered GRACE data
grace_data = pd.DataFrame(filtered_grace_data_list)

# Step 9: Filter GRACE data for grid points present in catalog data for the common year_month
grace_filtered_data = grace_data.merge(events_per_month[['lat_grid', 'lon_grid', 'year_month']],
                                       left_on=['latitude', 'longitude', 'year_month'],
                                       right_on=['lat_grid', 'lon_grid', 'year_month'],
                                       how='inner')

# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness']]
print("Filtered GRACE Data:")
print(grace_filtered_df)
grace_filtered_data = grace_data.merge(events_per_month[['lat_grid', 'lon_grid', 'year_month']],
                                       left_on=['latitude', 'longitude', 'year_month'],
                                       right_on=['lat_grid', 'lon_grid', 'year_month'],
                                       how='inner')

# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness']]
print("Filtered GRACE Data:")
print(grace_filtered_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-2-a37285d58a34>:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  catalog_data = pd.read_csv(file_path, delim_whitespace=True)


Number of unique grid points for each month:
    year_month  unique_lat_grid_count  unique_lon_grid_count
0      2002-04                      7                      8
1      2002-05                     15                     12
2      2002-06                      8                     10
3      2002-07                     11                     12
4      2002-08                      8                      9
..         ...                    ...                    ...
256    2023-08                     13                     15
257    2023-09                     12                     11
258    2023-10                     13                     12
259    2023-11                     11                      9
260    2023-12                     10                     10

[261 rows x 3 columns]
Grid with the maximum number of events for January 2020:
      lat_grid  lon_grid  event_count  \
2638      37.0    -121.5            4   

                                      lat_list  \
2638  [37

KeyError: 'latitude'

In [ ]:
grace_data

,year_month,latitude,longitude,lwe_thickness
0,2002-04,-89.75,0.25,-10.319427
1,2002-05,-89.75,0.25,-10.306927
2,2002-08,-89.75,0.25,-10.294414
3,2002-09,-89.75,0.25,-10.281900
4,2002-10,-89.75,0.25,-10.269387
...,...,...,...,...
59875195,2023-11,89.75,359.75,-2.173941
59875196,2023-12,89.75,359.75,-2.173941
59875197,2024-01,89.75,359.75,-2.173941
59875198,2024-02,89.75,359.75,-2.173941


In [ ]:
events_per_month

,lat_grid,lon_grid,year_month,event_count,lat_list,lon_list,max_mag,min_mag,avg_mag
172,36.5,-121.5,2022-04,1,[36.6781667],[-121.3041667],3.22,3.22,3.22
130,35.0,-117.0,2022-04,1,[35.0381667],[-116.9765],3.62,3.62,3.62
134,35.5,-118.5,2022-04,1,[35.3131667],[-118.5361667],3.28,3.28,3.28
35,32.5,-115.5,2022-04,1,[32.415],[-115.2528333],3.31,3.31,3.31
25,32.0,-117.0,2022-04,1,[31.9305],[-116.8925],4.53,4.53,4.53
...,...,...,...,...,...,...,...,...,...
108,34.0,-116.5,2023-12,2,"[34.0128333, 34.0125]","[-116.344, -116.3435]",3.25,3.05,3.15
142,35.5,-118.5,2023-12,1,[35.4003333],[-118.6451667],3.52,3.52,3.52
71,33.5,-118.0,2023-12,1,[33.6798333],[-117.9773333],3.43,3.43,3.43
199,37.0,-118.5,2023-12,1,[37.0655],[-118.5315],3.49,3.49,3.49


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the catalog data from Google Drive
file_path = '/content/drive/MyDrive/grace_data/catalog.txt'
catalog_data = pd.read_csv(file_path, delim_whitespace=True)

# OLD CODE START

# OLD CODE END

# New Step 1: Convert the date column to datetime format
catalog_data['date'] = pd.to_datetime(catalog_data['date'], format='%Y-%m-%d')

# Filter seismic data starting from 2022-04
catalog_data = catalog_data[catalog_data['date'] >= '2002-04-01']

# OLD CODE START
# New Step 2: Divide the dataset into a grid of 1 x 1 degree
# Round the latitude and longitude to the nearest 1 to create 1 x 1 degree grids
catalog_data['lat_grid'] = (catalog_data['latitude'] // 1 * 1).round(1)
# OLD CODE END

# New Step 2: Divide the dataset into a grid of 0.5 x 0.5 degree
# Round the latitude and longitude to the nearest 0.25 or 0.75
catalog_data['lat_grid'] = (catalog_data['latitude'] * 2).round() / 2
catalog_data['lon_grid'] = (catalog_data['longitude'] * 2).round() / 2

# New Step 3: Group by lat_grid, lon_grid, and month to get the sum of events per month
catalog_data['year_month'] = catalog_data['date'].dt.to_period('M')
events_per_month = catalog_data.groupby(['lat_grid', 'lon_grid', 'year_month']).agg(
    event_count=('year_month', 'size'),
    lat_list=('latitude', lambda x: list(x)),
    lon_list=('longitude', lambda x: list(x)),
    max_mag=('mag', 'max'),
    min_mag=('mag', 'min'),
    avg_mag=('mag', 'mean')
).reset_index()

# New Step 4: Sort the result by year_month
events_per_month = events_per_month.sort_values(by='year_month')

# Step 5: Count the number of unique grid points for each month
unique_grid_points_per_month = events_per_month.groupby('year_month')[['lat_grid', 'lon_grid']].nunique().reset_index()
unique_grid_points_per_month.rename(columns={'lat_grid': 'unique_lat_grid_count', 'lon_grid': 'unique_lon_grid_count'}, inplace=True)

# Display the number of unique grid points for each month
print("Number of unique grid points for each month:")
print(unique_grid_points_per_month)

# Step 6: Get the grid with the maximum number of events for 01-01-2020
max_event_date = '2020-01'
events_2020_01 = events_per_month[events_per_month['year_month'] == max_event_date]
max_event_grid = events_2020_01[events_2020_01['event_count'] == events_2020_01['event_count'].max()]

# Display the grid with the maximum number of events for January 2020
print("Grid with the maximum number of events for January 2020:")
print(max_event_grid[['lat_grid', 'lon_grid', 'event_count', 'lat_list', 'lon_list']])

# Display the result
events_per_month.head()

# New Step 7: Read a NetCDF file (GRACE Mascon data)
from netCDF4 import Dataset

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the GRACE Mascon data from Google Drive
netcdf_file_path = '/content/drive/MyDrive/grace_data/GRACE_Mascon_data.nc'
with Dataset(netcdf_file_path, mode='r') as nc_file:
    # List all variables in the NetCDF file
    print("Variables in GRACE Mascon NetCDF file:")
    print(nc_file.variables.keys())

    # Extract data for specific variables (example: lwe_thickness)
    lwe_thickness = nc_file.variables['lwe_thickness'][:]
    latitudes = nc_file.variables['lat'][:]
    longitudes = nc_file.variables['lon'][:]
    times = nc_file.variables['time'][:]

    # Display basic information about the extracted data
    print("LWE Thickness Data Shape:", lwe_thickness.shape)
    print("Latitude Data Shape:", latitudes.shape)
    print("Longitude Data Shape:", longitudes.shape)
    print("Time Data Shape:", times.shape)

    # Step 8: Add year_month to GRACE data
from datetime import datetime, timedelta

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Step 8: Filter GRACE data for California region only
california_lat_min, california_lat_max = 32, 42
california_lon_min, california_lon_max = -125, -114

# Filter GRACE data for California latitudes and longitudes
lat_indices = np.where((latitudes >= california_lat_min) & (latitudes <= california_lat_max))[0]
lon_indices = np.where((longitudes >= california_lon_min) & (longitudes <= california_lon_max))[0]

# Extract the filtered data
filtered_latitudes = latitudes[lat_indices]
filtered_longitudes = longitudes[lon_indices]
filtered_lwe_thickness = lwe_thickness[:, lat_indices, :][:, :, lon_indices]

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Create a DataFrame for filtered GRACE data
filtered_grace_data_list = []
for time_idx, year_month in enumerate(grace_year_month):
    for lat_idx, latitude in enumerate(filtered_latitudes):
        for lon_idx, longitude in enumerate(filtered_longitudes):
            lwe_value = filtered_lwe_thickness[time_idx, lat_idx, lon_idx]
            filtered_grace_data_list.append({
                'year_month': year_month,
                'latitude': latitude,
                'longitude': longitude,
                'lwe_thickness': lwe_value
            })

# Convert list of dictionaries to DataFrame
grace_data = pd.DataFrame(filtered_grace_data_list)

# Ensure correct column names for grace_data
# No need to rename columns, keep as 'latitude' and 'longitude'

# Step 9: Filter GRACE data for grid points present in catalog data for the common year_month
grace_filtered_data = grace_data.merge(events_per_month[['lat_grid', 'lon_grid', 'year_month']],
                                       left_on=['latitude', 'longitude', 'year_month'],
                                       right_on=['lat_grid', 'lon_grid', 'year_month'],
                                       how='inner')

# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness']]
print("Filtered GRACE Data:")
print(grace_filtered_df)
grace_filtered_data = grace_data.merge(events_per_month[['lat_grid', 'lon_grid', 'year_month']],
                                       left_on=['lat_grid', 'lon_grid', 'year_month'],
                                       right_on=['lat_grid', 'lon_grid', 'year_month'],
                                       how='inner')

# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness']]
print("Filtered GRACE Data:")
print(grace_filtered_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-10-edba5c51a872>:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  catalog_data = pd.read_csv(file_path, delim_whitespace=True)


Number of unique grid points for each month:
    year_month  unique_lat_grid_count  unique_lon_grid_count
0      2002-04                      7                      8
1      2002-05                     15                     12
2      2002-06                      8                     10
3      2002-07                     11                     12
4      2002-08                      8                      9
..         ...                    ...                    ...
256    2023-08                     13                     15
257    2023-09                     12                     11
258    2023-10                     13                     12
259    2023-11                     11                      9
260    2023-12                     10                     10

[261 rows x 3 columns]
Grid with the maximum number of events for January 2020:
      lat_grid  lon_grid  event_count  \
2638      37.0    -121.5            4   

                                      lat_list  \
2638  [37

KeyError: 'latitude'

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the catalog data from Google Drive
file_path = '/content/drive/MyDrive/grace_data/catalog.txt'
catalog_data = pd.read_csv(file_path, delim_whitespace=True)

# OLD CODE START

# OLD CODE END

# New Step 1: Convert the date column to datetime format
catalog_data['date'] = pd.to_datetime(catalog_data['date'], format='%Y-%m-%d')

# Filter seismic data starting from 2022-04
catalog_data = catalog_data[catalog_data['date'] >= '2002-04-01']

# OLD CODE START
# New Step 2: Divide the dataset into a grid of 1 x 1 degree
# Round the latitude and longitude to the nearest 1 to create 1 x 1 degree grids
catalog_data['lat_grid'] = (catalog_data['latitude'] // 1 * 1).round(1)
# OLD CODE END

# New Step 2: Divide the dataset into a grid of 0.5 x 0.5 degree
# Round the latitude and longitude to the nearest 0.25 or 0.75
catalog_data['lat_grid'] = (catalog_data['latitude'] * 2).round() / 2
catalog_data['lon_grid'] = (catalog_data['longitude'] * 2).round() / 2

# New Step 3: Group by lat_grid, lon_grid, and month to get the sum of events per month
catalog_data['year_month'] = catalog_data['date'].dt.to_period('M')
events_per_month = catalog_data.groupby(['lat_grid', 'lon_grid', 'year_month']).agg(
    event_count=('year_month', 'size'),
    lat_list=('latitude', lambda x: list(x)),
    lon_list=('longitude', lambda x: list(x)),
    max_mag=('mag', 'max'),
    min_mag=('mag', 'min'),
    avg_mag=('mag', 'mean')
).reset_index()

# New Step 4: Sort the result by year_month
events_per_month = events_per_month.sort_values(by='year_month')

# Step 5: Count the number of unique grid points for each month
unique_grid_points_per_month = events_per_month.groupby('year_month')[['lat_grid', 'lon_grid']].nunique().reset_index()
unique_grid_points_per_month.rename(columns={'lat_grid': 'unique_lat_grid_count', 'lon_grid': 'unique_lon_grid_count'}, inplace=True)

# Display the number of unique grid points for each month
#print("Number of unique grid points for each month:")
#print(unique_grid_points_per_month)

# Step 6: Get the grid with the maximum number of events for 01-01-2020
max_event_date = '2020-01'
events_2020_01 = events_per_month[events_per_month['year_month'] == max_event_date]
max_event_grid = events_2020_01[events_2020_01['event_count'] == events_2020_01['event_count'].max()]

# Display the grid with the maximum number of events for January 2020
#print("Grid with the maximum number of events for January 2020:")
#print(max_event_grid[['lat_grid', 'lon_grid', 'event_count', 'lat_list', 'lon_list']])

# Display the result
events_per_month.head()

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the GRACE Mascon data from Google Drive
netcdf_file_path = '/content/drive/MyDrive/grace_data/GRACE_Mascon_data.nc'
with Dataset(netcdf_file_path, mode='r') as nc_file:
    # List all variables in the NetCDF file
    print("Variables in GRACE Mascon NetCDF file:")
    print(nc_file.variables.keys())

    # Extract data for specific variables (example: lwe_thickness)
    lwe_thickness = nc_file.variables['lwe_thickness'][:]
    latitudes = nc_file.variables['lat'][:]
    longitudes = nc_file.variables['lon'][:]

    # Convert longitudes from [0, 360] to [-180, 180] range
    longitudes = np.where(longitudes > 180, longitudes - 360, longitudes)
    times = nc_file.variables['time'][:]

    # Display basic information about the extracted data
    print("LWE Thickness Data Shape:", lwe_thickness.shape)
    print("Latitude Data Shape:", latitudes.shape)
    print("Longitude Data Shape:", longitudes.shape)
    print("Time Data Shape:", times.shape)

    # Step 8: Add year_month to GRACE data
from datetime import datetime, timedelta

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Step 8: Filter GRACE data for California region only
california_lat_min, california_lat_max = 32, 42
california_lon_min, california_lon_max = -125, -114

# Filter GRACE data for California latitudes and longitudes
lat_indices = np.where((latitudes >= california_lat_min) & (latitudes <= california_lat_max))[0]
lon_indices = np.where((longitudes >= california_lon_min) & (longitudes <= california_lon_max))[0]

# Extract the filtered data
filtered_latitudes = latitudes[lat_indices]
filtered_longitudes = longitudes[lon_indices]
filtered_lwe_thickness = lwe_thickness[:, lat_indices, :][:, :, lon_indices]

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Create a DataFrame for filtered GRACE data
filtered_grace_data_list = []
for time_idx, year_month in enumerate(grace_year_month):
    for lat_idx, latitude in enumerate(filtered_latitudes):
        for lon_idx, longitude in enumerate(filtered_longitudes):
            lwe_value = filtered_lwe_thickness[time_idx, lat_idx, lon_idx]
            filtered_grace_data_list.append({
                'year_month': year_month,
                'latitude': latitude,
                'longitude': longitude,
                'lwe_thickness': lwe_value
            })

# Convert list of dictionaries to DataFrame
grace_data = pd.DataFrame(filtered_grace_data_list)

# Ensure correct column names for grace_data
# No need to rename columns, keep as 'latitude' and 'longitude'

# Step 9: Display columns for debugging
print("GRACE Data Columns:")
print(grace_data.columns.tolist())
print("Events per Month Columns:")
print(events_per_month.columns.tolist())
print(events_per_month.columns)

# Step 9: Based on the printed column names, adjust the merge operation
grace_filtered_data = grace_data.merge(events_per_month,
                                       left_on=['latitude', 'longitude', 'year_month'],
                                       right_on=['lat_grid', 'lon_grid', 'year_month'],
                                       how='inner')

# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness']]
print("Filtered GRACE Data:")
print(grace_filtered_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-33-9bf8b1fa7f07>:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  catalog_data = pd.read_csv(file_path, delim_whitespace=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Variables in GRACE Mascon NetCDF file:
dict_keys(['lon', 'lat', 'time', 'land_mask', 'lwe_thickness', 'lon_bounds', 'lat_bounds', 'time_bounds'])
LWE Thickness Data Shape: (231, 360, 720)
Latitude Data Shape: (360,)
Longitude Data Shape: (720,)
Time Data Shape: (231,)
GRACE Data Columns:
['year_month', 'latitude', 'longitude', 'lwe_thickness']
Events per Month Columns:
['lat_grid', 'lon_grid', 'year_month', 'event_count', 'lat_list', 'lon_list', 'max_mag', 'min_mag', 'avg_mag']
Index(['lat_grid', 'lon_grid', 'year_month', 'event_count', 'lat_list',
       'lon_list', 'max_mag', 'min_mag', 'avg_mag'],
      dtype='object')
Filtered GRACE Data:
Empty DataFrame
Columns: [latitude, longitude, year_month, lwe_thickness]
Index: []


In [ ]:
filtered_grace_data_list

[{'year_month': Period('2002-04', 'M'),
  'latitude': 32.25,
  'longitude': -124.75,
  'lwe_thickness': -0.9047952296037399},
 {'year_month': Period('2002-04', 'M'),
  'latitude': 32.25,
  'longitude': -124.25,
  'lwe_thickness': -0.9047952296037399},
 {'year_month': Period('2002-04', 'M'),
  'latitude': 32.25,
  'longitude': -123.75,
  'lwe_thickness': -1.1805797916683298},
 {'year_month': Period('2002-04', 'M'),
  'latitude': 32.25,
  'longitude': -123.25,
  'lwe_thickness': -1.1805797916683298},
 {'year_month': Period('2002-04', 'M'),
  'latitude': 32.25,
  'longitude': -122.75,
  'lwe_thickness': -1.5233513287323492},
 {'year_month': Period('2002-04', 'M'),
  'latitude': 32.25,
  'longitude': -122.25,
  'lwe_thickness': -1.5233513287323492},
 {'year_month': Period('2002-04', 'M'),
  'latitude': 32.25,
  'longitude': -121.75,
  'lwe_thickness': -1.5233513287323492},
 {'year_month': Period('2002-04', 'M'),
  'latitude': 32.25,
  'longitude': -121.25,
  'lwe_thickness': -1.80683029914

In [ ]:
np.where(1321 > 180, 1 - 360, 1)

array(-359)

In [ ]:
events_per_month

,lat_grid,lon_grid,year_month,event_count,lat_list,lon_list,max_mag,min_mag,avg_mag
2817,37.5,-119.0,2002-04,1,[37.5148333],[-118.8278333],3.03,3.03,3.030
3721,39.5,-123.5,2002-04,1,[39.469],[-123.3263333],3.38,3.38,3.380
328,32.0,-116.0,2002-04,1,[31.8211667],[-116.2233333],3.16,3.16,3.160
3419,39.0,-123.0,2002-04,5,"[38.7846667, 38.8225, 38.8445, 38.7908333, 39....","[-122.7586667, -122.798, -122.8128333, -122.77...",3.98,3.13,3.528
1691,34.5,-116.5,2002-04,1,[34.524],[-116.2946667],4.39,4.39,4.390
...,...,...,...,...,...,...,...,...,...
2310,36.0,-117.5,2023-12,2,"[35.9288333, 35.9253333]","[-117.6818333, -117.684]",3.32,3.18,3.250
1737,35.0,-121.0,2023-12,1,[35.2355],[-120.784],3.07,3.07,3.070
1533,34.0,-116.5,2023-12,2,"[34.0128333, 34.0125]","[-116.344, -116.3435]",3.25,3.05,3.150
1042,33.0,-115.5,2023-12,2,"[33.1655, 33.1616667]","[-115.657, -115.6563333]",3.31,3.12,3.215


In [34]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
!pip install netCDF4


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the catalog data from Google Drive
file_path = '/content/drive/MyDrive/grace_data/catalog.txt'
catalog_data = pd.read_csv(file_path, delim_whitespace=True)

# OLD CODE START

# OLD CODE END

# New Step 1: Convert the date column to datetime format
catalog_data['date'] = pd.to_datetime(catalog_data['date'], format='%Y-%m-%d')

# Filter seismic data starting from 2022-04
catalog_data = catalog_data[catalog_data['date'] >= '2002-04-01']
catalog_data = catalog_data[catalog_data['mag'] >= 3]

# New Step 2: Divide the dataset into a grid of 0.5 x 0.5 degree
# Round the latitude and longitude to the nearest 0.25 or 0.5
catalog_data['lat_grid'] = np.where((catalog_data['latitude'] % 1) < 0.5,
                                    np.floor(catalog_data['latitude']) + 0.25,
                                    np.floor(catalog_data['latitude']) + 0.75)

catalog_data['lon_grid'] = np.where((catalog_data['longitude'] % 1) < 0.5,
                                     np.floor(catalog_data['longitude']) + 0.25,
                                     np.floor(catalog_data['longitude']) + 0.75)



# New Step 3: Group by lat_grid, lon_grid, and month to get the sum of events per month
catalog_data['year_month'] = catalog_data['date'].dt.to_period('M')
events_per_month = catalog_data.groupby(['lat_grid', 'lon_grid', 'year_month']).agg(
    event_count=('year_month', 'size'),
    lat_list=('latitude', lambda x: list(x)),
    lon_list=('longitude', lambda x: list(x)),
    max_mag=('mag', 'max'),
    min_mag=('mag', 'min'),
    avg_mag=('mag', 'mean')
).reset_index()

# New Step 4: Sort the result by year_month
events_per_month = events_per_month.sort_values(by='year_month')

# Step 5: Count the number of unique grid points for each month
unique_grid_points_per_month = events_per_month.groupby('year_month')[['lat_grid', 'lon_grid']].nunique().reset_index()
unique_grid_points_per_month.rename(columns={'lat_grid': 'unique_lat_grid_count', 'lon_grid': 'unique_lon_grid_count'}, inplace=True)

# Display the number of unique grid points for each month
print("Number of unique grid points for each month:")
print(unique_grid_points_per_month)

# Step 6: Get the grid with the maximum number of events for 01-01-2020
max_event_date = '2020-01'
events_2020_01 = events_per_month[events_per_month['year_month'] == max_event_date]
max_event_grid = events_2020_01[events_2020_01['event_count'] == events_2020_01['event_count'].max()]

# Display the grid with the maximum number of events for January 2020
print("Grid with the maximum number of events for January 2020:")
print(max_event_grid[['lat_grid', 'lon_grid', 'event_count', 'lat_list', 'lon_list']])

# Display the result
events_per_month.head()

# New Step 7: Read a NetCDF file (GRACE Mascon data)
from netCDF4 import Dataset

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the GRACE Mascon data from Google Drive
netcdf_file_path = '/content/drive/MyDrive/grace_data/GRACE_Mascon_data.nc'
with Dataset(netcdf_file_path, mode='r') as nc_file:
    # List all variables in the NetCDF file
    print("Variables in GRACE Mascon NetCDF file:")
    print(nc_file.variables.keys())

    # Extract data for specific variables (example: lwe_thickness)
    lwe_thickness = nc_file.variables['lwe_thickness'][:]
    latitudes = nc_file.variables['lat'][:]
    longitudes = nc_file.variables['lon'][:]

    # Convert longitudes from [0, 360] to [-180, 180] range
    longitudes = np.where(longitudes > 180, longitudes - 360, longitudes)
    times = nc_file.variables['time'][:]

    # Display basic information about the extracted data
    print("LWE Thickness Data Shape:", lwe_thickness.shape)
    print("Latitude Data Shape:", latitudes.shape)
    print("Longitude Data Shape:", longitudes.shape)
    print("Time Data Shape:", times.shape)

    # Step 8: Add year_month to GRACE data
from datetime import datetime, timedelta

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Step 8: Filter GRACE data for California region only
california_lat_min, california_lat_max = 31, 42
california_lon_min, california_lon_max = -125, -114

# Filter GRACE data for California latitudes and longitudes
lat_indices = np.where((latitudes >= california_lat_min) & (latitudes <= california_lat_max))[0]
lon_indices = np.where((longitudes >= california_lon_min) & (longitudes <= california_lon_max))[0]

# Extract the filtered data
filtered_latitudes = latitudes[lat_indices]
filtered_longitudes = longitudes[lon_indices]
filtered_lwe_thickness = lwe_thickness[:, lat_indices, :][:, :, lon_indices]

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Create a DataFrame for filtered GRACE data
filtered_grace_data_list = []
for time_idx, year_month in enumerate(grace_year_month):
    for lat_idx, latitude in enumerate(filtered_latitudes):
        for lon_idx, longitude in enumerate(filtered_longitudes):
            lwe_value = filtered_lwe_thickness[time_idx, lat_idx, lon_idx]
            filtered_grace_data_list.append({
                'year_month': year_month,
                'latitude': latitude,
                'longitude': longitude,
                'lwe_thickness': lwe_value
            })

# Convert list of dictionaries to DataFrame
grace_data = pd.DataFrame(filtered_grace_data_list)

# Ensure correct column names for grace_data
# No need to rename columns, keep as 'latitude' and 'longitude'

# Step 9: Display columns for debugging
print("GRACE Data Columns:")
print(grace_data.columns.tolist())
print("Events per Month Columns:")
print(events_per_month.columns.tolist())
print(events_per_month.columns)

# Step 9: Based on the printed column names, adjust the merge operation
grace_filtered_data = grace_data.merge(events_per_month,
                                       left_on=['latitude', 'longitude', 'year_month'],
                                       right_on=['lat_grid', 'lon_grid', 'year_month'],
                                       how='inner')

# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness']]
print("Filtered GRACE Data:")
print(grace_filtered_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-34-656aaaa9ef27>:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  catalog_data = pd.read_csv(file_path, delim_whitespace=True)


Number of unique grid points for each month:
    year_month  unique_lat_grid_count  unique_lon_grid_count
0      2002-04                      7                      8
1      2002-05                     12                     10
2      2002-06                      8                      9
3      2002-07                     10                     12
4      2002-08                      9                     10
..         ...                    ...                    ...
256    2023-08                     12                     14
257    2023-09                     10                     11
258    2023-10                     12                     12
259    2023-11                     10                     10
260    2023-12                     10                      7

[261 rows x 3 columns]
Grid with the maximum number of events for January 2020:
      lat_grid  lon_grid  event_count                          lat_list  \
2380     36.75   -121.75            3  [36.9616667, 36.842, 36.8761

In [35]:
import keras_tuner as kt

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model

ModuleNotFoundError: No module named 'keras_tuner'

In [38]:
def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=50):
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=50, restore_best_weights=True)
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    history = model.fit(train_set, validation_data=valid_set, epochs=epochs,
                        callbacks=[early_stopping_cb])
    valid_loss, valid_mae = model.evaluate(valid_set)
    return valid_mae

In [39]:
# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness', 'event_count', 'max_mag', 'min_mag', 'avg_mag']]


# Step 10: Prepare data for LSTM model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Extract predictors and target
X = grace_filtered_df[['event_count', 'year_month','max_mag', 'min_mag', 'avg_mag','lwe_thickness']]
y = grace_filtered_df[['year_month','lwe_thickness']]

X = X.set_index('year_month')

X_train = X.loc['2002-04':'2013-05', ['event_count', 'max_mag', 'avg_mag']]
X_valid = X.loc['2013-06':'2020-05', ['event_count', 'max_mag', 'avg_mag']]
X_test = X.loc['2020-06':'2024-01', ['event_count', 'max_mag', 'avg_mag']]

y_train = X.loc['2002-04':'2013-05', 'lwe_thickness']
y_valid = X.loc['2013-06':'2020-05', 'lwe_thickness']

tf.random.set_seed(42)
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    X_train.to_numpy(),
    targets=y_train,
    sequence_length=1,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    X_valid.to_numpy(),
    targets=y_valid,
    sequence_length=1,
    batch_size=32
)



deep_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 3]),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1)
])


fit_and_evaluate(deep_model, train_ds , valid_ds , 1e-4 )
"""
# Scale the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Reshape X for LSTM input (samples, timesteps, features)
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split data into training and testing sets
train_size = int(len(X_lstm) * 0.8)
X_train, X_test = X_lstm[:train_size], X_lstm[train_size:]
y_train, y_test = y_scaled[:train_size], y_scaled[train_size:]
"""
#,'max_mag', 'min_mag', 'avg_mag','lwe_thickness'

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - loss: 5.3657 - mae: 5.8437 - val_loss: 15.3107 - val_mae: 15.8061
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 5.4113 - mae: 5.8893 - val_loss: 15.3039 - val_mae: 15.7994
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 5.3918 - mae: 5.8698 - val_loss: 15.2972 - val_mae: 15.7927
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 5.3359 - mae: 5.8138 - val_loss: 15.2906 - val_mae: 15.7861
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 5.3366 - mae: 5.8149 - val_loss: 15.2840 - val_mae: 15.7795
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 5.3250 - mae: 5.8033 - val_loss: 15.2776 - val_mae: 15.7732
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 5.2255 - mae: 5.7035 - val_loss: 15.2707 - val_mae: 15.7663
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 5.3738 - mae: 5.8506 - val_loss: 15.2647 - val_mae: 15.7603
Epoch 9/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/s

'\n# Scale the data\nscaler_X = MinMaxScaler()\nscaler_y = MinMaxScaler()\nX_scaled = scaler_X.fit_transform(X)\ny_scaled = scaler_y.fit_transform(y.reshape(-1, 1))\n\n# Reshape X for LSTM input (samples, timesteps, features)\nX_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))\n\n# Split data into training and testing sets\ntrain_size = int(len(X_lstm) * 0.8)\nX_train, X_test = X_lstm[:train_size], X_lstm[train_size:]\ny_train, y_test = y_scaled[:train_size], y_scaled[train_size:]\n'

In [ ]:
grace_filtered_df

,latitude,longitude,year_month,lwe_thickness,event_count,max_mag,min_mag,avg_mag
0,31.75,-116.25,2002-04,-0.007192,1,3.16,3.16,3.160000
1,32.25,-115.25,2002-04,2.824223,3,3.30,3.01,3.203333
2,32.25,-114.75,2002-04,2.007574,1,4.10,4.10,4.100000
3,34.75,-116.25,2002-04,5.708782,1,4.39,4.39,4.390000
4,36.75,-121.75,2002-04,4.652653,2,3.60,3.11,3.355000
...,...,...,...,...,...,...,...,...
3684,35.75,-117.75,2023-12,-37.412506,3,3.42,3.18,3.306667
3685,36.75,-120.75,2023-12,-39.046023,1,3.70,3.70,3.700000
3686,37.25,-118.75,2023-12,-36.929195,1,3.49,3.49,3.490000
3687,38.75,-122.75,2023-12,-20.952386,2,3.50,3.41,3.455000


In [ ]:
for elem in train_ds:
  print(elem)

(<tf.Tensor: shape=(32, 1, 3), dtype=float64, numpy=
array([[[  2.        ,   3.38      ,   3.255     ]],

       [[  1.        ,   3.07      ,   3.07      ]],

       [[472.        ,   7.2       ,   3.45082627]],

       [[  1.        ,   4.4       ,   4.4       ]],

       [[  1.        ,   3.81      ,   3.81      ]],

       [[  1.        ,   3.9       ,   3.9       ]],

       [[  1.        ,   3.        ,   3.        ]],

       [[  1.        ,   3.19      ,   3.19      ]],

       [[  1.        ,   3.25      ,   3.25      ]],

       [[  3.        ,   3.43      ,   3.28      ]],

       [[  1.        ,   3.47      ,   3.47      ]],

       [[  2.        ,   3.79      ,   3.475     ]],

       [[  1.        ,   3.35      ,   3.35      ]],

       [[  2.        ,   4.1       ,   3.75      ]],

       [[  1.        ,   3.4       ,   3.4       ]],

       [[  1.        ,   3.27      ,   3.27      ]],

       [[  1.        ,   3.55      ,   3.55      ]],

       [[  1.        ,   3.5 

In [ ]:

"""

# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness', 'event_count', 'max_mag', 'min_mag', 'avg_mag']]
print("Filtered GRACE Data:")
print(grace_filtered_df)

# Step 10: Prepare data for LSTM model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Extract predictors and target
X = grace_filtered_df[['event_count', 'max_mag', 'min_mag', 'avg_mag']].values
y = grace_filtered_df['lwe_thickness'].values

# Scale the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Reshape X for LSTM input (samples, timesteps, features)
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split data into training and testing sets
train_size = int(len(X_lstm) * 0.8)
X_train, X_test = X_lstm[:train_size], X_lstm[train_size:]
y_train, y_test = y_scaled[:train_size], y_scaled[train_size:]

# Step 11: Build and train LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Fit the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Step 12: Make predictions
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Display predictions
print("Predicted LWE values:")
print(y_pred)

# Step 13: Plot the graph for predicted and actual LWE values
plt.figure(figsize=(10, 6))
plt.plot(range(len(y_test)), scaler_y.inverse_transform(y_test), label='Actual LWE', color='b')
plt.plot(range(len(y_pred)), y_pred, label='Predicted LWE', color='r')
plt.xlabel('Time Step')
plt.ylabel('LWE Thickness')
plt.title('Actual vs Predicted LWE Thickness')
plt.legend()
plt.show()
"""